In [18]:

import numpy as np
import torch


# load the data
X = np.load('data/x.npy')
y = np.load('data/y.npy')

# y = np.array([get_fen_score(board) for board in X])


# print(X[0])

# reshape y to be [n_samples, 1]
y = y.reshape(-1, 1)


# preview the data
print(X.shape)
print(y.shape)



# split the data into train and test
X_train = X[:int(len(X) * 0.8)]
y_train = y[:int(len(y) * 0.8)]
X_test = X[int(len(X) * 0.8):]
y_test = y[int(len(y) * 0.8):]

# convert to torch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

(2030, 12, 8, 8)
(2030, 1)


In [19]:
# data preview
print(X_train.shape)
print(y_train.shape)



torch.Size([1624, 12, 8, 8])
torch.Size([1624, 1])


# autoencoder arhitecture network

we define a network with increasingly narrower layers. Middle layer is the embedding.

In [60]:
import torch.functional as F
import torch
import torch.nn as nn

relu = F.F.relu


# define the neural network
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # autoencoder start with input size 
        self.fc1 = nn.Linear(12*8*8, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, 256)
        self.dropout3 = nn.Dropout(0.5)
        # embedding layer
        self.fc4 = nn.Linear(256, 128)
        self.dropout4 = nn.Dropout(0.5)
        # decoder
        self.fc5 = nn.Linear(128,256)
        self.dropout5 = nn.Dropout(0.5)
        self.fc6 = nn.Linear(256, 512)
        self.dropout6 = nn.Dropout(0.5)
        self.fc7 = nn.Linear(512, 1024)
        self.dropout7 = nn.Dropout(0.5)
        self.fc8 = nn.Linear(1024, 12*8*8)



    def forward(self, x):
        # reshape
        x = x.view(-1, 12*8*8)
        # pass
        x = relu(self.fc1(x))
        x = self.dropout1(x)
        x = relu(self.fc2(x))
        x = self.dropout2(x)
        x = relu(self.fc3(x))
        x = self.dropout3(x)
        x = relu(self.fc4(x))
        x = self.dropout4(x)
        x = relu(self.fc5(x))
        x = self.dropout5(x)
        x = relu(self.fc6(x))
        x = self.dropout6(x)
        x = relu(self.fc7(x))
        x = self.dropout7(x)
        x = self.fc8(x)
        # reshape
        return x.view(-1, 12, 8, 8)
    
    def predict(self, x):
        # reshape
        x = x.view(-1, 12*8*8)
        # pass
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = relu(self.fc3(x))
        x = relu(self.fc4(x))
        x = relu(self.fc5(x))
        x = relu(self.fc6(x))
        x = relu(self.fc7(x))
        x = self.fc8(x)
        # reshape
        return x.view(-1, 12, 8, 8)
    
    def embed(self, x):
        # reshape
        x = x.view(-1, 12*8*8)

        # forward pass
        x = relu(self.fc1(x))
        x = relu(self.fc2(x))
        x = relu(self.fc3(x))
        x = relu(self.fc4(x))
        return x

# running the model

In [63]:
net = Net()

BATCH_SIZE = 200

def myLoss (outputs, inputs):
    diff = torch.square(outputs - inputs)
    diff = torch.sum(diff,axis=(1,2,3)) # sum between features

    diff = torch.mean(diff, axis=(0)) # mean between samples
    return diff

# criterion = torch.nn.L1Loss( reduction='mean', reduce=True)

# define the optimizer
optimizer = torch.optim.Adagrad(net.parameters(), lr=0.007)

for epoch in range(20):
    for batch in range(0, len(X_train), BATCH_SIZE):
        optimizer.zero_grad()
        outputs = net.forward(X_train)
        loss = myLoss(outputs, X_train) # difference between outputs and inputs
        loss.backward()
        optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 16.637657165527344
epoch 1, loss 15.141633987426758
epoch 2, loss 14.2717866897583
epoch 3, loss 13.798789978027344
epoch 4, loss 13.68751049041748
epoch 5, loss 13.421772956848145
epoch 6, loss 13.292110443115234
epoch 7, loss 13.251688957214355
epoch 8, loss 13.22873306274414
epoch 9, loss 13.144010543823242
epoch 10, loss 13.05517864227295
epoch 11, loss 12.990988731384277
epoch 12, loss 13.012935638427734
epoch 13, loss 12.8754301071167
epoch 14, loss 12.89731216430664
epoch 15, loss 12.852861404418945
epoch 16, loss 12.802620887756348
epoch 17, loss 12.759516716003418
epoch 18, loss 12.74320125579834
epoch 19, loss 12.736407279968262


In [67]:
# test the model
with torch.no_grad():
    outputs = net.predict(X_test)
    loss = myLoss(outputs, X_test)
    print('test loss: {}'.format((loss.item())))

# save the model
MODEL_PATH = 'model_material.pth'
torch.save(net.state_dict(), MODEL_PATH)

test loss: 12.830560684204102


In [44]:
# load the model
net = Net()
net.load_state_dict(torch.load(MODEL_PATH))

# test the model
with torch.no_grad():
    outputs = net.predict(X_test)
    loss = criterion(outputs, y_test)
    print('test loss: {}'.format(abs(loss.item())))

test loss: 0.863443911075592
